# Analysis

### About the database

Dataset of flight cancellations and delays for US airlines in January 2023, extracted from the US Department of Transportation, Bureau of Transportation Statistics (https://www.transstats.bts.gov) and available on Kaggle.

Variables include flight routes (origin and destination), time intervals between events (minutes, time at destination), reasons/attributions for delays and cancellations.

### Variable description

#### DOT_CODE_DICTIONARY Table

| Variable | Description |
| -------- | --------- |
| Code | Numeric identifier from the U.S. Department of Transportation (DOT) for airlines |
| Description | Airline description |

#### AIRLINE_CODE_DICTIONARY Table

| Variable | Description |
| -------- | --------- |
| Code | Unique operator code for aircraft operator agencies |
| Description | Description of the aircraft operator agency |

#### flights_202301 Table

| Variable | Description |
| -------- | --------- |
| FL_DATE | Flight date (yyyymmdd) |
| AIRLINE_CODE | Unique operator code. When multiple operators use the same code, a numeric suffix is used to differentiate previous users, e.g., PA, PA(1), PA(2). |
| DOT_CODE | Identification number assigned by the US DOT to identify a unique airline (carrier). A unique airline (carrier) is defined as one that owns and reports under the same DOT certificate regardless of its code, name, or holding/company. |
| FL_NUMBER | Flight number |
| ORIGIN | Origin airport |
| ORIGIN_CITY | Origin airport, city name |
| DEST | Destination airport |
| DEST_CITY | Destination airport, city name |
| CRSDEPTIME | Scheduled departure time recorded in CRS (Computer Reservation System) (local time: hhmm) |
| DEP_TIME | Actual departure time (local time: hhmm) |
| DEP_DELAY | Difference in minutes between scheduled and actual departure time. Early departures show negative numbers. |
| TAXI_OUT | Taxi out time in minutes (taxi is the process of moving an airplane while it is on the runway) |
| WHEELS_OFF | Exact takeoff time (local time: hhmm) |
| WHEELS_ON | Exact landing time (local time: hhmm) |
| TAXI_IN | Taxi in time in minutes |
| CRSARRTIME | Scheduled arrival time recorded in CRS (local time: hhmm) |
| ARR_TIME | Actual arrival time (local time: hhmm) |
| ARR_DELAY | Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers. |
| CANCELLED | Flight cancelled indicator (1=Yes) |
| CANCELLATION_CODE | Specifies the reason for cancellation |
| DIVERTED | Flight diverted indicator (1=Yes) |
| CRSELAPSEDTIME | Total elapsed flight time in minutes recorded in CRS |
| ELAPSED_TIME | Total actual elapsed flight time in minutes |
| AIR_TIME | Air time in minutes |
| DISTANCE | Distance between airports (miles) |
| DELAYDUECARRIER | Carrier delay in minutes |
| DELAYDUEWEATHER | Weather delay in minutes |
| DELAYDUENAS | National Air System delay in minutes |
| DELAYDUESECURITY | Security delay in minutes |
| DELAYDUELATE_AIRCRAFT | Late aircraft delay in minutes |

## 1. Imports

In [1]:
import pandas as pd
from datetime import time

In [ ]:
df_dot_code_dictionary = pd.read_excel("../dataset/DOT_CODE_DICTIONARY.xlsx")
df_airline_code_dictionary = pd.read_csv("../dataset/AIRLINE_CODE_DICTIONARY.csv")
df_flights_202301 = pd.read_csv("../dataset/flights_202301.csv")

## 2. Data treatment

In [ ]:
# Renaming columns to create a pattern
df_dot_code_dictionary = df_dot_code_dictionary.rename(
    columns=lambda col: f"DOT_{col}" if col != "Code" else col
)
df_airline_code_dictionary = df_airline_code_dictionary.rename(
    columns=lambda col: f"AIRLINE_{col}" if col != "Code" else col
)

In [ ]:
print(f"Shape of df_flights_202301: {df_flights_202301.shape}")
print(f"Shape of df_dot_code_dictionary: {df_dot_code_dictionary.shape}")
print(f"Shape of df_airline_code_dictionary: {df_airline_code_dictionary.shape}")

Shape de df_voos202301: (538837, 33)
Shape de df_DotCodeDictionary: (1737, 2)
Shape de df_AirlineCodeDictionary: (1729, 2)


### a. Converting types in df_flights_202301

In [ ]:
df_flights_202301.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538837 entries, 0 to 538836
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   FL_DATE                  538837 non-null  object 
 1   AIRLINE_CODE             538837 non-null  object 
 2   DOT_CODE                 538837 non-null  int64  
 3   FL_NUMBER                538837 non-null  int64  
 4   ORIGIN                   538837 non-null  object 
 5   ORIGIN_CITY              538837 non-null  object 
 6   DEST                     538837 non-null  object 
 7   DEST_CITY                538837 non-null  object 
 8   CRS_DEP_TIME             538837 non-null  int64  
 9   DEP_TIME                 528859 non-null  float64
 10  DEP_DELAY                528855 non-null  float64
 11  TAXI_OUT                 528640 non-null  float64
 12  WHEELS_OFF               528640 non-null  float64
 13  WHEELS_ON                528318 non-null  float64
 14  TAXI

In [ ]:
def optimize_flight_data_types(df):
    df_optimized = df.copy()

    # 1. DATE CONVERSION - more robust
    df_optimized["FL_DATE"] = pd.to_datetime(
        df_optimized["FL_DATE"],
        format="%Y-%m-%d",
        errors='coerce'
    )

    # 2. CATEGORICAL COLUMNS - with memory optimization
    cols_category = [
        "AIRLINE_CODE", "ORIGIN", "ORIGIN_CITY",
        "DEST", "DEST_CITY", "CANCELLATION_CODE"
    ]
    for col in cols_category:
        df_optimized[col] = df_optimized[col].astype("category")

    # 3. BOOLEAN VARIABLES
    df_optimized["CANCELLED"] = df_optimized["CANCELLED"].astype(bool)
    df_optimized["DIVERTED"] = df_optimized["DIVERTED"].astype(bool)

    # 4. IMPROVED FUNCTION FOR TIME CONVERSION AND ALL TIME COLUMNS
    def hhmm_to_time_optimized(val):
        """Convert HHMM format to time, with robust error handling"""
        if pd.isna(val):
            return pd.NaT
        try:
            val = int(val)
            if val == 2400:
                val = 0
            elif val > 2400:
                return pd.NaT
            hh = val // 100
            mm = val % 100
            if hh > 23 or mm > 59:
                return pd.NaT
            return time(hh, mm)
        except (ValueError, TypeError):
            return pd.NaT
    time_columns = [
        "CRS_DEP_TIME", "DEP_TIME", "WHEELS_OFF", "WHEELS_ON",
        "CRS_ARR_TIME", "ARR_TIME"
    ]
    for col in time_columns:
        if col in df_optimized.columns:
            df_optimized[col] = df_optimized[col].apply(hhmm_to_time_optimized)

    # 5. NUMERIC TYPE OPTIMIZATION
    # Columns that can be converted to smaller int (memory saving)
    int_columns_small = ["FL_NUMBER", "DOT_CODE", "FL_YEAR", "FL_MONTH", "FL_DAY"]
    for col in int_columns_small:
        if col in df_optimized.columns:
            max_val = df_optimized[col].max()
            if max_val <= 32767:  # int16
                df_optimized[col] = df_optimized[col].astype("int16")
            elif max_val <= 2147483647:  # int32
                df_optimized[col] = df_optimized[col].astype("int32")
    # Distance can likely be int16 (max ~32k miles)
    if "DISTANCE" in df_optimized.columns:
        df_optimized["DISTANCE"] = df_optimized["DISTANCE"].astype("int16")
    # Convert to float32 (saves 50% memory vs float64)
    float_columns = [
        "DEP_DELAY", "TAXI_OUT", "TAXI_IN", "ARR_DELAY",
        "CRS_ELAPSED_TIME", "ELAPSED_TIME", "AIR_TIME",
        "DELAY_DUE_CARRIER", "DELAY_DUE_WEATHER", "DELAY_DUE_NAS",
        "DELAY_DUE_SECURITY", "DELAY_DUE_LATE_AIRCRAFT"
    ]
    for col in float_columns:
        if col in df_optimized.columns:
            df_optimized[col] = pd.to_numeric(df_optimized[col], errors='coerce')
            df_optimized[col] = df_optimized[col].astype("float32")

    return df_optimized

In [ ]:
df_flights_optimized = optimize_flight_data_types(df_flights_202301)

In [8]:
df_flights_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538837 entries, 0 to 538836
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   FL_DATE                  538837 non-null  datetime64[ns]
 1   AIRLINE_CODE             538837 non-null  category      
 2   DOT_CODE                 538837 non-null  int16         
 3   FL_NUMBER                538837 non-null  int16         
 4   ORIGIN                   538837 non-null  category      
 5   ORIGIN_CITY              538837 non-null  category      
 6   DEST                     538837 non-null  category      
 7   DEST_CITY                538837 non-null  category      
 8   CRS_DEP_TIME             538837 non-null  object        
 9   DEP_TIME                 528859 non-null  object        
 10  DEP_DELAY                528855 non-null  float32       
 11  TAXI_OUT                 528640 non-null  float32       
 12  WHEELS_OFF      

### b. Converting types of df_airline_code_dictionary

In [ ]:
df_airline_code_dictionary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Code                 1728 non-null   object
 1   AIRLINE_Description  1729 non-null   object
dtypes: object(2)
memory usage: 27.1+ KB


In [ ]:
df_airline_optimized = df_airline_code_dictionary.copy()

In [ ]:
# 1. Convert 'Code' to category
df_airline_optimized['Code'] = df_airline_optimized['Code'].astype('category')

# 2. Convert 'AIRLINE_Description' to category
df_airline_optimized['AIRLINE_Description'] = df_airline_optimized['AIRLINE_Description'].astype('category')

In [12]:
df_airline_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Code                 1728 non-null   category
 1   AIRLINE_Description  1729 non-null   category
dtypes: category(2)
memory usage: 163.0 KB


### c. Converting types of df_dot_code_dictionary

In [ ]:
df_dot_code_dictionary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Code             1737 non-null   int64 
 1   DOT_Description  1733 non-null   object
dtypes: int64(1), object(1)
memory usage: 27.3+ KB


In [ ]:
df_dot_optimized = df_dot_code_dictionary.copy()

In [ ]:
# 1. Convert to numeric type to ensure conversion works
df_dot_optimized['Code'] = pd.to_numeric(df_dot_optimized['Code'], errors='coerce')

# Check max value to choose best integer type
max_dot_code = df_dot_optimized['Code'].max()

if max_dot_code <= 32767:  # int16 limit
    df_dot_optimized['Code'] = df_dot_optimized['Code'].astype('int16')
elif max_dot_code <= 2147483647:  # int32 limit
    df_dot_optimized['Code'] = df_dot_optimized['Code'].astype('int32')

# 2. Convert 'DOT_Description' to category
df_dot_optimized['DOT_Description'] = df_dot_optimized['DOT_Description'].astype('category')

In [16]:
df_dot_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Code             1737 non-null   int16   
 1   DOT_Description  1733 non-null   category
dtypes: category(1), int16(1)
memory usage: 85.0 KB


### d. Checking optimized sizes

In [ ]:
def compare_memory_usage(df_name, df_original, df_optimized):
    mem_original = df_original.memory_usage(deep=True).sum() / 1024**2
    mem_optimized = df_optimized.memory_usage(deep=True).sum() / 1024**2

    print(f"Memory improvement analysis for **{df_name}**")
    print(f"Original DataFrame: {mem_original:.2f} MB")
    print(f"Optimized DataFrame: {mem_optimized:.2f} MB")
    print(f"Savings: {mem_original - mem_optimized:.2f} MB ({((mem_original - mem_optimized) / mem_original * 100):.1f}%)")
    print("=" * 50)

compare_memory_usage("df_flights_202301", df_flights_202301, df_flights_optimized)
compare_memory_usage("df_airline_code_dictionary", df_airline_code_dictionary, df_airline_optimized)
compare_memory_usage("df_dot_code_dictionary", df_dot_code_dictionary, df_dot_optimized)

Analise de melhora de **df_voos202301**
DataFrame original: 306.27 MB
DataFrame otimizado: 191.10 MB
Economia: 115.17 MB (37.6%)
Analise de melhora de **df_AirlineCodeDictionary**
DataFrame original: 0.20 MB
DataFrame otimizado: 0.33 MB
Economia: -0.13 MB (-66.2%)
Analise de melhora de **df_DotCodeDictionary**
DataFrame original: 0.14 MB
DataFrame otimizado: 0.19 MB
Economia: -0.06 MB (-41.2%)


### Merge data

Main: df_flights_202301

df_airline_code_dictionary: Code <> df_flights_202301: AIRLINE_CODE

df_dot_code_dictionary: Code <> df_flights_202301: DOT_CODE

In [ ]:
df_flights_202301["AIRLINE_CODE"].unique()

array(['9E', 'AA', 'AS', 'B6', 'DL', 'F9', 'G4', 'HA', 'MQ', 'NK', 'WN',
       'YX', 'OH', 'OO', 'UA'], dtype=object)

In [ ]:
df_flights_202301["DOT_CODE"].unique()

array([20363, 19805, 19930, 20409, 19790, 20436, 20368, 19690, 20398,
       20416, 19393, 20452, 20397, 20304, 19977])

In [ ]:
df_merged = pd.merge(
    df_flights_202301,
    df_dot_code_dictionary,
    left_on="DOT_CODE",
    right_on="Code",
    how="left"
).drop(columns=["Code"])

# Merge with AIRLINE_CODE
df_merged = pd.merge(
    df_merged,
    df_airline_code_dictionary,
    left_on="AIRLINE_CODE",
    right_on="Code",
    how="left"
).drop(columns=["Code"])

In [21]:
df_merged.head()

,FL_DATE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,DEP_TIME,...,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,FL_YEAR,FL_MONTH,FL_DAY,DOT_Description,AIRLINE_Description
0,2023-01-02,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,757.0,...,NaN,NaN,NaN,NaN,NaN,2023,1,2,Endeavor Air Inc.: 9E,Endeavor Air Inc.
1,2023-01-03,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,755.0,...,NaN,NaN,NaN,NaN,NaN,2023,1,3,Endeavor Air Inc.: 9E,Endeavor Air Inc.
2,2023-01-04,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,755.0,...,NaN,NaN,NaN,NaN,NaN,2023,1,4,Endeavor Air Inc.: 9E,Endeavor Air Inc.
3,2023-01-05,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,754.0,...,NaN,NaN,NaN,NaN,NaN,2023,1,5,Endeavor Air Inc.: 9E,Endeavor Air Inc.
4,2023-01-06,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,759.0,...,NaN,NaN,NaN,NaN,NaN,2023,1,6,Endeavor Air Inc.: 9E,Endeavor Air Inc.


In [22]:
df_merged.shape

(538837, 35)